Célula 1: Instalações e Importações

In [8]:
# --- CÉLULA 1: Instalações e Importações (CORRIGIDA) ---

# Descomente e execute esta linha se você não tiver as bibliotecas instaladas
# !pip install pandas numpy scikit-learn "imblearn" pyyaml joblib

import pandas as pd
import numpy as np
import yaml
import joblib
import os  # <--- ADICIONE ESTA LINHA
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import warnings

warnings.filterwarnings('ignore', category=FutureWarning)

# Carregar o arquivo de configuração
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

print("Bibliotecas e configuração carregadas com sucesso!")

Bibliotecas e configuração carregadas com sucesso!


Célula 2: Carregar, Limpar e Unificar TODOS os Datasets

In [2]:
# --- CÉLULA 2: Carregar, Limpar e Unificar TODOS os Datasets ---

all_cleaned_dfs = []
datasets_to_process = ['koi', 'k2', 'tess']

print("Iniciando o carregamento e limpeza de todos os datasets...")

for name in datasets_to_process:
    print(f"\n--- Processando: {name.upper()} ---")
    
    file_path = config['datasets'][name]['file_path']
    try:
        df = pd.read_csv(file_path, comment='#')
    except FileNotFoundError:
        print(f"AVISO: Arquivo para '{name}' não encontrado em '{file_path}'. Pulando.")
        continue
    
    dataset_config = config['datasets'][name]
    target_column = dataset_config['target_column']
    mapping = dataset_config['disposition_mapping']
    
    df.columns = df.columns.str.lower().str.replace('[. ]', '_', regex=True)
    target_col_std = target_column.lower().replace(' ', '_').replace('.', '_')
    
    if target_col_std in df.columns:
        df = df.rename(columns={target_col_std: 'disposition'})
        df['disposition'] = df['disposition'].fillna('Unknown').map(mapping).fillna('Unknown')
        valid_dispositions = ['Confirmed', 'Candidate', 'False Positive']
        df = df[df['disposition'].isin(valid_dispositions)]
        
        all_cleaned_dfs.append(df)
        print(f"'{name}' limpo e adicionado. Shape: {df.shape}")
    else:
        print(f"AVISO: Coluna alvo '{target_col_std}' não encontrada em '{name}'. Pulando.")

# Unificar todos os DataFrames limpos em um só
combined_df = pd.concat(all_cleaned_dfs, ignore_index=True)

print("\n" + "="*50)
print("TODOS OS DATASETS FORAM UNIFICADOS!")
print(f"Shape final do DataFrame combinado: {combined_df.shape}")
print("\nDistribuição de classes no dataset combinado:")
print(combined_df['disposition'].value_counts())

Iniciando o carregamento e limpeza de todos os datasets...

--- Processando: KOI ---
'koi' limpo e adicionado. Shape: (9564, 141)

--- Processando: K2 ---
AVISO: Coluna alvo 'pl_disposition' não encontrada em 'k2'. Pulando.

--- Processando: TESS ---
'tess' limpo e adicionado. Shape: (6921, 87)

TODOS OS DATASETS FORAM UNIFICADOS!
Shape final do DataFrame combinado: (16485, 224)

Distribuição de classes no dataset combinado:
disposition
Candidate         7120
False Positive    6036
Confirmed         3329
Name: count, dtype: int64


Célula 3: Preparação das Features (X) e do Alvo (y)

In [3]:
# --- CÉLULA 3: Preparação das Features (X) e do Alvo (y) ---

y = combined_df['disposition']
df_for_features = combined_df.drop(columns=['disposition'])
X = df_for_features.select_dtypes(include=np.number)

id_cols_to_drop = ['kepid', 'kepoi_name', 'kepler_name', 'rowid', 'tic_id', 'toi']
cols_to_drop_found = [col for col in id_cols_to_drop if col in X.columns]
if cols_to_drop_found:
    X = X.drop(columns=cols_to_drop_found)

print("Separação em Features (X) e Alvo (y) concluída.")
print(f"Shape de X (features): {X.shape}")
print(f"Shape de y (alvo): {y.shape}")

Separação em Features (X) e Alvo (y) concluída.
Shape de X (features): (16485, 200)
Shape de y (alvo): (16485,)


Célula 4: Divisão em Dados de Treino e Teste

In [4]:
# --- CÉLULA 4: Divisão em Treino e Teste ---

test_size = config['preprocessing']['test_size']
random_state = config['preprocessing']['random_state']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=test_size, 
    random_state=random_state, 
    stratify=y
)

print("Divisão em treino e teste concluída.")
print(f"Shape de X_train: {X_train.shape} | Shape de y_train: {y_train.shape}")
print(f"Shape de X_test: {X_test.shape}   | Shape de y_test: {y_test.shape}")

Divisão em treino e teste concluída.
Shape de X_train: (13188, 200) | Shape de y_train: (13188,)
Shape de X_test: (3297, 200)   | Shape de y_test: (3297,)


Célula 5: Pipeline de Transformação e Balanceamento

In [5]:
# --- CÉLULA 5: Pipeline de Transformação e Balanceamento ---

imputer = KNNImputer(n_neighbors=config['preprocessing']['knn_neighbors'])
scaler = StandardScaler()
label_encoder = LabelEncoder()
smote = SMOTE(random_state=config['app']['random_state'])

print("--- PROCESSANDO DADOS DE TREINO ---")
X_train_imputed = imputer.fit_transform(X_train)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_imputed, y_train)
X_train_processed = scaler.fit_transform(X_train_balanced)
y_train_processed = label_encoder.fit_transform(y_train_balanced)
print("Dados de treino processados!")

print("\n--- PROCESSANDO DADOS DE TESTE ---")
X_test_imputed = imputer.transform(X_test)
X_test_processed = scaler.transform(X_test_imputed)
y_test_processed = label_encoder.transform(y_test)
print("Dados de teste processados!")

print("\n" + "="*50)
print("Verificação Final do Processamento:")
print(f"Shape final do X de treino: {X_train_processed.shape}")
print(f"Shape final do X de teste:  {X_test_processed.shape}")
print(f"Classes aprendidas: {label_encoder.classes_} -> {np.unique(y_train_processed)}")

--- PROCESSANDO DADOS DE TREINO ---
Dados de treino processados!

--- PROCESSANDO DADOS DE TESTE ---
Dados de teste processados!

Verificação Final do Processamento:
Shape final do X de treino: (17088, 169)
Shape final do X de teste:  (3297, 169)
Classes aprendidas: ['Candidate' 'Confirmed' 'False Positive'] -> [0 1 2]


Célula 6: Treinamento e Avaliação do "Super-Modelo"

In [6]:
# --- CÉLULA 6: Treinamento e Avaliação do "Super-Modelo" ---

# Usaremos RandomForest, um modelo robusto e popular
model = RandomForestClassifier(n_estimators=100, random_state=config['app']['random_state'], n_jobs=-1)

print("Iniciando o treinamento do modelo RandomForest com todos os dados...")
model.fit(X_train_processed, y_train_processed)
print("Treinamento concluído!")

print("\nFazendo previsões no conjunto de teste...")
y_pred = model.predict(X_test_processed)

print(f"\nAcurácia do Super-Modelo: {accuracy_score(y_test_processed, y_pred):.2%}")
print("\nRelatório de Classificação Detalhado:")
print(classification_report(y_test_processed, y_pred, target_names=label_encoder.classes_))

Iniciando o treinamento do modelo RandomForest com todos os dados...
Treinamento concluído!

Fazendo previsões no conjunto de teste...

Acurácia do Super-Modelo: 86.99%

Relatório de Classificação Detalhado:
                precision    recall  f1-score   support

     Candidate       0.82      0.91      0.86      1424
     Confirmed       0.88      0.82      0.85       666
False Positive       0.94      0.85      0.89      1207

      accuracy                           0.87      3297
     macro avg       0.88      0.86      0.87      3297
  weighted avg       0.88      0.87      0.87      3297



Célula 7: Salvar o Modelo e os Transformadores ("Artefatos")

In [9]:
# --- CÉLULA 7: Salvar o Modelo e os Transformadores ---

# Cria uma pasta para salvar os artefatos, se não existir
os.makedirs('artifacts', exist_ok=True)

joblib.dump(model, 'artifacts/exoplanet_super_model.pkl')
joblib.dump(scaler, 'artifacts/super_scaler.pkl')
joblib.dump(label_encoder, 'artifacts/super_label_encoder.pkl')
joblib.dump(imputer, 'artifacts/super_imputer.pkl')
joblib.dump(list(X.columns), 'artifacts/X_columns.pkl')

print("Super-modelo, transformadores e lista de colunas foram salvos com sucesso na pasta 'artifacts/'!")

Super-modelo, transformadores e lista de colunas foram salvos com sucesso na pasta 'artifacts/'!
